In [200]:
import re
import math
from collections import defaultdict
from datetime import datetime
import pandas as pd
from timeit import default_timer as timer
import numpy as np
import tqdm

In [205]:
TITLE = re.compile(r'title:\s*(.*)')
REV =  re.compile(r'rating:\s*(\d+)\s* votes')
COS = re.compile(r'cutomer:\s*(\w+)')
ID = re.compile(r'Id:\s*(\d+)')
CATEGORY = re.compile(r'[\s]*\|([a-zA-Z ]+)\[[\d]+\]\|')
REVIEW = re.compile(r'[\d]{4}-[\d]{1,2}-[\d]{1,2}[\s]+cutomer:[\s]+([\w\d]+)[\s]+rating:[\s]([\d]+)')

ASIN =re.compile( r'ASIN:\s*\d+')
CID = re.compile(r'cutomer:\s*(\w+)')
DISC = re.compile(r'discontinued product')
CSIZE = re.compile(r'reviews total\s*(\d+)')

In [199]:
product_reviews = {}
df = pd.DataFrame() 
f = open('amazon-meta.txt', 'r',encoding="utf8")
data = open('data.csv', 'w')

l = f.readlines()
write_to_file = False
cate = False

data.write('{};{};{};{}\n'.format('prod', 'cust', 'mark', 'cat'))
for l in tqdm.tqdm(l[7:]):
    l: str = l.strip()
    #pusta linia
    if not l:
        write_to_file = False
        category = None
        continue
        
    #czytanie id
    if l.startswith('Id:'):
        matched = ID.search(l)
        product_id = matched.groups()[0]
        continue
    
    #czytanie tytulu
    if l.startswith('title:'):
        matched = TITLE.search(l)
        product_title = matched.groups()[0]
        continue
    
    #ma kategorie
    if l.startswith('categories'):
        is_category_block = True
        continue

    #ma reviews
    if l.startswith('reviews'):
        write_to_file = True
        cate = False
        continue

    #czytanie kategorii
    if cate and not category:
        matched = CATEGORY_PATTERN.search(line)
        if matched:
            cats = matched.groups()[0]
            if cats != 'Categories':
                category = matched.groups()[0]
        continue

    if write_to_file:
        matched = REVIEW.search(line)
        user_id, review = matched.groups()
        data.write('{};{};{};{}\n'.format(product_id, user_id, review, category))

100%|██████████████████████████████████████████████████████████████████| 15010567/15010567 [01:00<00:00, 248770.74it/s]


In [383]:
df = pd.read_csv('data_final.csv', sep=';')

In [384]:
df

,product_id,customer_id,review,category
0,1,A2JW67OY8U6HHK,5,Books
1,1,A2VE83MZF98ITY,5,Books
2,2,A11NCO6YTE4BTJ,5,Books
3,2,A9CQ3PLRNIR83,4,Books
4,2,A13SG9ACZ9O5IM,5,Books
...,...,...,...,...
7593239,548549,A2PD80S1N7920J,4,Books
7593240,548550,A3NKS7CVEJVTQ6,5,DVD
7593241,548550,A3EQ4YAZ5OEVK9,5,DVD
7593242,548550,A3HRK7C6LL7CRI,5,DVD


In [387]:
def sort(df= pd.DataFrame, category = 'Books', users= 100, products = 10, sort_user = False):
    #wybranie jednej kategorii
    one_cat = df[df.category == category]
    #wybranie produktow ktore maja najwiecej reviews
    naj_prod = one_cat[one_cat.product_id.isin(one_cat.groupby(by='product_id').count().sort_values('review', ascending=False).index[:products].tolist())]
    
    #znalezenie osob ktore daly najwiecej revsow
    ids_most_active_users_of_category = naj_prod.groupby('customer_id').count()
    
    #posortowanie ludzi z najwieksza iloscia revsow
    if sort_user:
        ids_most_active_users_of_category = ids_most_active_users_of_category.sort_values('review', ascending=False)
    
    #pivot na tabelce
    pivot_table = pd.pivot_table(naj_prod[naj_prod.customer_id.isin(ids_most_active_users_of_category.index[:users].tolist())], values=['review'], columns=['product_id'], index=['customer_id'], aggfunc='first', fill_value=0)
    rates = pivot_table['review'].values
    return rates

In [392]:
rates = sort(df, category='Books', users=10, products=10, sort_user=False)

In [393]:
rates

array([[0, 4, 4, 4, 4, 4, 0, 0, 4, 4],
       [0, 5, 5, 5, 5, 5, 0, 0, 5, 5],
       [4, 0, 0, 0, 0, 0, 4, 4, 0, 0],
       [4, 0, 0, 0, 0, 0, 4, 4, 0, 0],
       [0, 5, 5, 5, 5, 5, 0, 0, 5, 5],
       [5, 0, 0, 0, 0, 0, 5, 5, 0, 0],
       [4, 0, 0, 0, 0, 0, 4, 4, 0, 0],
       [0, 5, 5, 5, 5, 5, 0, 0, 5, 5],
       [5, 0, 0, 0, 0, 0, 5, 5, 0, 0],
       [4, 0, 0, 0, 0, 0, 4, 4, 0, 0]], dtype=int64)